## data import + clean

In [36]:
import pandas as pd
import numpy as np
# import random as rn
import sklearn
# from scipy import stats
# import math
import re

from sklearn.preprocessing import  LabelEncoder #OneHotEncoder
from sklearn import ensemble

from sklearn.model_selection import GridSearchCV,cross_val_score
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error,accuracy_score

# from sklearn.ensemble import RandomForestClassifier
from sklearn import linear_model

In [13]:
totDF = pd.read_csv('../../data/processed/Cleaned_Data_Set.csv')

In [108]:
sample_per_year = 2000

dwnSmplDF = totDF.groupby('birth_year',group_keys = False).apply(lambda x: x.sample(sample_per_year))

In [111]:
r1 = re.compile('.*reporting')
r2 = re.compile('.*imputed')

cols_to_drop1 = list(filter((r1.match), totDF.columns))
cols_to_drop2 = list(filter((r2.match), totDF.columns))
cols_to_drop3 = ['admit_NICU']
cols_to_drop = cols_to_drop1 + cols_to_drop2 + cols_to_drop3

cols_to_keep = [col for col in totDF.columns if col not in cols_to_drop]

X = dwnSmplDF[cols_to_keep].copy()
target = dwnSmplDF[['admit_NICU']].copy()
# X = totDF[cols_to_keep].copy()
# target = totDF[['admit_NICU']].copy()

numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
catDF = X.select_dtypes(include=object).copy()
numDF = X.select_dtypes(include=numerics).copy() #only numeric columns

le = LabelEncoder()
catDF = catDF.apply(le.fit_transform)
encoded_target = target.apply(le.fit_transform)

cl_df = pd.concat([numDF,catDF],axis=1)

## Random Forest

In [113]:
randomForest = ensemble.RandomForestClassifier()
grid_para_forest = [{
	'n_estimators': np.linspace(50,int(np.sqrt(len(cl_df))),10,dtype=int),
	'min_samples_split' : [100,10,2],
	'min_samples_leaf' : [100,10,1]
}]
randomForest.set_params(random_state=108)
grid_search_forest = GridSearchCV(randomForest, grid_para_forest, scoring='accuracy', cv=5, n_jobs=-1)
%time grid_search_forest.fit(cl_df, encoded_target) #put in the df here

/opt/anaconda3/lib/python3.7/site-packages/sklearn/model_selection/_search.py:715: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  self.best_estimator_.fit(X, y, **fit_params)


CPU times: user 3.98 s, sys: 281 ms, total: 4.26 s
Wall time: 1min 24s


GridSearchCV(cv=5, error_score='raise-deprecating',
             estimator=RandomForestClassifier(bootstrap=True, class_weight=None,
                                              criterion='gini', max_depth=None,
                                              max_features='auto',
                                              max_leaf_nodes=None,
                                              min_impurity_decrease=0.0,
                                              min_impurity_split=None,
                                              min_samples_leaf=1,
                                              min_samples_split=2,
                                              min_weight_fraction_leaf=0.0,
                                              n_estimators='warn', n_jobs=None,
                                              oob_score=False, random_state=108,
                                              verbose=0, warm_start=False),
             iid='warn', n_jobs=-1,
             param_grid=[{'

In [132]:
print(
    '''Random Forest
sample size: {0}
best param : {1}
best score : {2}
r2         : {3}'''\
      .format(len(encoded_target), \
              grid_search_forest.best_params_,\
              grid_search_forest.best_score_, \
              r2_score(encoded_target, grid_search_forest.predict(cl_df)))
     )

Random Forest
sample size: 10000
best param : {'min_samples_leaf': 1, 'min_samples_split': 2, 'n_estimators': 100}
best score : 0.9491
r2         : 1.0


## GLM

In [102]:
logit_1 = linear_model.LogisticRegression()
logit_1.set_params(C=1e4, n_jobs=-1)
%time logit_1.fit(cl_df, encoded_target)

/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/opt/anaconda3/lib/python3.7/site-packages/sklearn/utils/validation.py:724: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:469: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)
/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:1544: UserWarning: 'n_jobs' > 1 does not have any effect when 'solver' is set to 'liblinear'. Got 'n_jobs' = 8.
  " = {}.".format(effective_n_jobs(self.n_jobs)))


CPU times: user 11min 32s, sys: 10.6 s, total: 11min 43s
Wall time: 11min 54s


LogisticRegression(C=10000.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='warn', n_jobs=-1, penalty='l2',
                   random_state=None, solver='warn', tol=0.0001, verbose=0,
                   warm_start=False)

In [88]:
print('sample size: {0} \nscore: {1}\nr2: {2}'.format(len(encoded_target), \
                                                      logit_1.score(cl_df, encoded_target), \
                                                      r2_score(encoded_target, logit_1.predict(cl_df))))

sample size: 10000 
score: 0.9473
r2: 0.3049125488527581


In [92]:
print('sample size: {0} \nscore: {1}\nr2: {2}'.format(len(encoded_target), \
                                                      logit_1.score(cl_df, encoded_target), \
                                                      r2_score(encoded_target, logit_1.predict(cl_df))))

sample size: 25000 
score: 0.94044
r2: 0.2847605560292703


In [96]:
print('sample size: {0} \nscore: {1}\nr2: {2}'.format(len(encoded_target), \
                                                      logit_1.score(cl_df, encoded_target), \
                                                      r2_score(encoded_target, logit_1.predict(cl_df))))

sample size: 50000 
score: 0.94196
r2: 0.28236519003706706


In [100]:
print('sample size: {0} \nscore: {1}\nr2: {2}'.format(len(encoded_target), \
                                                      logit_1.score(cl_df, encoded_target), \
                                                      r2_score(encoded_target, logit_1.predict(cl_df))))

sample size: 100000 
score: 0.93952
r2: 0.2710406424346249


In [103]:
print('sample size: {0} \nscore: {1}\nr2: {2}'.format(len(encoded_target), \
                                                      logit_1.score(cl_df, encoded_target), \
                                                      r2_score(encoded_target, logit_1.predict(cl_df))))
#11min 54sec

sample size: 1000000 
score: 0.940666
r2: 0.27437829660503155
